# Automatic Differentiation: From Scratch

## Introduction
As discussed in our presntation and the reading, calculating derivatives is central to machine learning. We generally categorize differentiation into four methods:

1.  Manual Differentiation: Tedious and error-prone.
2.  Symbolic Differentiation: Operates on mathematical expressions. Can lead to "expression swell" (exponentially large formulas).
3.  Numerical Differentiation: Uses finite differences $\left( \dfrac{f(x+h) - f(x)}{h} \right)$. Prone to truncation and round-off errors.
4.  Automatic Differentiation (AD): Applies symbolic rules at the elementary operation level while keeping intermediate numerical results. 

In this notebook, we will implement symbolic and numeric differentiation ato see their problems, and then implement forward mode (using dual numbers) and reverse mode (by building a computational graph) to understand how deep learning frameworks compute gradients under the hood.

In [11]:
import math

## Symbolic Differentiation & Its Limitations

Before diving into Automatic Differentiation (AD), it is crucial to understand why we don't simply use the methods taught in Calculus I (Symbolic Differentiation) or simple approximations (Numerical Differentiation).

Symbolic differentiation transforms a mathematical expression into a new expression for the derivative using rules like the product rule: 

$$ \frac{d}{dx}(f(x)g(x)) = f'(x)g(x) + f(x)g'(x) $$

While exact, this method struggles with two main issues discussed in the course readings:
1.  Expression Swell: The derivative expression can become exponentially larger than the original function.
2.  Control Flow: Symbolic differentiation requires a closed-form expression. It fails (or becomes incredibly complex) when code involves `if` statements, loops, or recursion.

#### Exercise 1: The Control Flow Problem
Below is a Python function `strange_relu` that involves control flow. 

Your task: Manually implement its derivative, `symbolic_grad_strange_relu`. 

*Note how you must manually replicate the logic (the `if` statement) of the primal function to calculate the derivative correctly.*

In [ ]:
def strange_relu(x):
    # A piecewise function that symbolic engines struggle to simplify generically
    if x >= 2.0:
        return x ** 3
    elif x > 0.0:
        return x ** 2
    else:
        return 0.0

def symbolic_grad_strange_relu(x):
    """
    TODO: Manually implement the exact derivative of the function above.
    You will need to manually mirror the control flow.
    """
    # YOUR CODE HERE

In [13]:
# Test your symbolic implementation
test_points = [-1.0, 1.0, 3.0]
print("x\t f(x)\t f'(x)")
print("----------------------")
for x in test_points:
    print(f"{x}\t {strange_relu(x)}\t {symbolic_grad_strange_relu(x)}")

print("""Expected:
-1.0\t 0.0\t 0.0
1.0\t 1.0\t 2.0
3.0\t 27.0\t 27.0""")

x	 f(x)	 f'(x)
----------------------
-1.0	 0.0	 0.0
1.0	 1.0	 2.0
3.0	 27.0	 27.0
Expected:
-1.0	 0.0	 0.0
1.0	 1.0	 2.0
3.0	 27.0	 27.0


## Numerical Differentiation (Finite Differences)

Numerical differentiation approximates the derivative using the definition of a limit, but with a fixed, small step size $h$. 

The Forward Difference is defined as:
$$ f'(x) \approx \frac{f(x+h) - f(x)}{h} $$

However, as noted in the PDF (Section 2.1, Figure 3), this method is prone to two types of errors:
1.  Truncation Error: Mathematical error because $h \neq 0$.
2.  Round-off Error: Floating point issues when subtracting two nearly identical numbers ($f(x+h)$ and $f(x)$).

We can reduce truncation error by using the Centered Difference:
$$ f'(x) \approx \frac{f(x+h) - f(x-h)}{2h} $$

#### Exercise 2: Comparing Finite Differences
Your task: Implement both the forward and centered difference methods below and compare their accuracy against the exact derivative.

In [ ]:
def forward_diff(f, x, h=1e-5):
    return (f(x + h) - f(x)) / h

def centered_diff(f, x, h=1e-5):
    """
    TODO: Implement the centered difference formula.
    """
    # YOUR CODE HERE

In [15]:
x_val = 3.0
h_val = 1e-4

exact = symbolic_grad_strange_relu(x_val) 
fwd = forward_diff(strange_relu, x_val, h_val)
cnt = centered_diff(strange_relu, x_val, h_val)

print(f"Exact Derivative at x={x_val}: {exact}")
print(f"Forward Difference (h={h_val}): {fwd:.6f} (Error: {abs(exact-fwd):.2e})")
print(f"Centered Difference (h={h_val}): {cnt:.6f} (Error: {abs(exact-cnt):.2e})")

Exact Derivative at x=3.0: 27.0
Forward Difference (h=0.0001): 27.000900 (Error: 9.00e-04)
Centered Difference (h=0.0001): 27.000000 (Error: 1.01e-08)


## Forward Mode AD and Dual Numbers

Forward mode AD can be implemented using dual numbers. A dual number is represented as $a + \dot{a}\epsilon$, where:
*   $a$ is the primal (the value of the variable).
*   $\dot{a}$ is the tangent (the derivative of the variable).
*   $\epsilon$ is a "nilpotent number," meaning $\epsilon^2 = 0$.

Dual number arithmetic mirrors differentiation, for example,
1.  Addition: $(a + \dot{a}\epsilon) + (b + \dot{b}\epsilon) = (a+b) + (\dot{a}+\dot{b})\epsilon$
2.  Multiplication: $(a + \dot{a}\epsilon)(b + \dot{b}\epsilon) = ab + (a\dot{b} + \dot{a}b)\epsilon$

### Exercise 3: Implementing Dual Numbers

Below is a class `DualNumber`. The addition and multiplication methods have been implemented for you. 

Your Task: Implement the `__pow__` method (for power $x^k$) and the `sin` function using the rules of dual numbers and differentials.

In [ ]:
class DualNumber:
    def __init__(self, primal, deriv=0.0):
        self.primal = primal
        self.deriv = deriv
        
    def __str__(self):
        return f"DualNumber(Value={self.primal:.4f}, Derivative={self.deriv:.4f})"

    def __repr__(self):
        return self.__str__()

    def __add__(self, other):
        # Handle adding a scalar constant
        if isinstance(other, (int, float)):
            return DualNumber(self.primal + other, self.deriv)
        
        # Handle adding two Dual Numbers
        return DualNumber(self.primal + other.primal, self.deriv + other.deriv)

    def __sub__(self, other):
        # Handle subtracting a scalar constant
        if isinstance(other, (int, float)):
            return DualNumber(self.primal - other, self.deriv)
        
        # Handle subtracting two Dual Numbers
        return DualNumber(self.primal - other.primal, self.deriv - other.deriv)

    def __truediv__(self, other):
        # Handle dividing by scalar constant
        if isinstance(other, (int, float)):
            return DualNumber(self.primal / other, self.deriv / other)
        
        # Quotient Rule: (f'g - fg') / g^2 (Dividing two Dual Numbers)
        new_primal = self.primal / other.primal
        new_deriv = (self.deriv * other.primal - self.primal * other.deriv) / (other.primal ** 2)
        return DualNumber(new_primal, new_deriv)
    
    def __rtruediv__(self, other):
        # Handle scalar constant divided by Dual Number
        if isinstance(other, (int, float)):
            new_primal = other / self.primal
            new_deriv = (-other * self.deriv) / (self.primal ** 2)
            return DualNumber(new_primal, new_deriv)
        
        raise NotImplementedError("Right division only implemented for scalar constants.")

    def __mul__(self, other):
        # Handle multiplying by scalar constant
        if isinstance(other, (int, float)):
            return DualNumber(self.primal * other, self.deriv * other)
        
        # Product Rule: f(x)g'(x) + g(x)f'(x) (Multiplying two Dual Numbers)
        new_primal = self.primal * other.primal
        new_deriv = (self.primal * other.deriv) + (self.deriv * other.primal)
        return DualNumber(new_primal, new_deriv)
    
    def __neg__(self):
        # Handle negation of Dual Number
        return DualNumber(-self.primal, -self.deriv)






    def __pow__(self, power):
        """
        TODO: Implement the power rule for Dual Numbers.
        Recall: d/dx (u(x)^n) = n * u(x)^(n-1) * u'(x)
        Note: Assume 'power' is a scalar integer/float, not a DualNumber.
        """
        
        pass # YOUR CODE HERE
    
    
    
    
    

def dual_sin(x):
    """
    TODO: Implement sine for a DualNumber.
    Recall: d/dx sin(u(x)) = cos(u(x)) * u'(x)
    """
    if isinstance(x, (int, float)):
        return math.sin(x)
    
    pass # YOUR CODE HERE






def dual_log(x):
    # Provided for the test function later
    if isinstance(x, (int, float)):
        return math.log(x)
    return DualNumber(math.log(x.primal), x.deriv / x.primal)

## The Jacobian with Forward Mode

Forward mode computes the Jacobian one column at a time. 

If we have a function $f: \mathbb{R}^n \to \mathbb{R}^m$, we need $n$ passes to compute the full gradient/Jacobian. 

### Exercise 4: Compute the full gradient

We'll use the same function from the reading: $$y= f(x_{1},x_{2}) = \ln(x_{1})+x_{1}x_{2}-\sin(x_{2})$$

Your task: Using the `DualNumber` class above and the `test_function` below, write a script to compute the gradient $\nabla f = \left[\dfrac{\partial f}{\partial x_1}, \dfrac{\partial f}{\partial x_2}\right]$ at $(2, 5)$.

In [ ]:
def test_function(x1, x2):
    # f(x1, x2) = ln(x1) + x1*x2 - sin(x2)
    return dual_log(x1) + (x1 * x2) - dual_sin(x2)

# TODO: Calculate df/dx1 and df/dx2. 
# Hint: You need to initialize x1 and x2 similarly, but their dual number components should differ. Use the DualNumber class.

# Pass 1: df/dx1

# YOUR CODE HERE

grad_x1 = None  # Replace

# Pass 2: df/dx2

# YOUR CODE HERE

grad_x2 = None  # Replace

print(f"Gradient: [{grad_x1}, {grad_x2}]")
print("Expected:   [5.5, 1.716...]")


Gradient: [5.5, 1.7163378145367738]
Expected:   [5.5, 1.716...]


## Reverse Mode AD (Graph-Based)

Forward mode is efficient for $n \ll m$ (few inputs, many outputs). However, in deep learning, we usually have a scalar loss function and millions of inputs (weights), so $n \gg m$. 

For this, we use reverse mode AD, which builds a computational graph (Primal Trace) and then propagates derivatives backwards (Adjoint Trace). This allows us to compute the gradient of a scalar output with respect to *all* inputs in a single backward pass.

### Exercise 5: Graph Construction
We will define a `Value` class that stores:
1. `data`: The numerical value.
2. `grad`: The derivative of the final output with respect to this value (the adjoint $\bar{v}$).
3. `_backward`: A function that calculates the local gradients and sends them to the parents.

This architecture allows us to define-by-run (dynamic computational graph), dealing with control flow naturally.

Your task: implement the `relu` and `_backward` methods for our `Value` class

In [ ]:
class Value:
    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op

    def __repr__(self):
        return f"Value(data={self.data:.4f}, grad={self.grad:.4f})"

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            # Addition distributes gradient equally: d/dx (x+y) = 1
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            # Product rule: d/dx (xy) = y, d/dy (xy) = x
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out

    def relu(self):
        """
        TODO: Implement the ReLU (Rectified Linear Unit) operation.
        Forward: x if x > 0 else 0
        Backward: 1 * grad if x > 0 else 0
        
        This demonstrates AD's ability to handle control flow/logic.
        
        Hint: inside _backward, you need to update self.grad. 
        Recall that self.grad accumulates the gradients from its children. 
        For ReLU, if the original value was greater than 0, let the gradient (out.grad) pass through. 
        If it was 0 or less, "kill" the gradient (multiply by 0)
        """
        
        # 1. Compute the forward value
        out_data = None # YOUR CODE HERE
        
        # 2. Create the output Node
        out = Value(out_data, (self,), 'ReLU')

        # 3. Define the backward function closure
        def _backward():
            # YOUR CODE HERE
            pass

        out._backward = _backward
        return out

    def backward(self):
        # Topological sort to ensure we calculate gradients in correct order
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        
        # Set gradient of the output (loss) to 1.0
        self.grad = 1.0
        # Apply chain rule in reverse order
        for node in reversed(topo):
            node._backward()

In [ ]:
# Case 1: Active ReLU
a = Value(2.0)
b = Value(3.0)
c = Value(-1.0)

z = a * b + c  # 2*3 - 1 = 5
y = z.relu()   # ReLU(5) = 5

y.backward()
print("--- Case 1 (Positive) ---")
print(f"Output y:\t {y.data}\t (Expected: 5.0)")
print(f"Grad a:  \t {a.grad}\t (Expected: 3.0)")

# Case 2: Inactive ReLU
a2 = Value(2.0)
b2 = Value(-3.0)
c2 = Value(-1.0)

z2 = a2 * b2 + c2 # 2*-3 - 1 = -7
y2 = z2.relu()    # ReLU(-7) = 0

y2.backward()
print("\n--- Case 2 (Negative) ---")
print(f"Output y:\t {y2.data}\t (Expected: 0.0)")
print(f"Grad a:  \t {a2.grad}\t (Expected: 0.0)")

--- Case 1 (Positive) ---
Output y:	 5.0	 (Expected: 5.0)
Grad a:  	 3.0	 (Expected: 3.0)

--- Case 2 (Negative) ---
Output y:	 0	 (Expected: 0.0)
Grad a:  	 0.0	 (Expected: 0.0)
